<a href="https://colab.research.google.com/github/file00/PyTorch/blob/main/Optimizing_Model_Parameters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Optimizing Model Parameters

*   Detailed Theoretical Explanation: https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html

### Prerequisite Code

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

100%|██████████| 26421880/26421880 [00:00<00:00, 90235811.14it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 5327719.17it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw




100%|██████████| 4422102/4422102 [00:00<00:00, 51288004.57it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 23068672.00it/s]


Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



### Hyperparameters

In [2]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

### Optimization Loop

### Loss Function

In [3]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

### Optimizer

In [4]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

### Full Implementation

In [5]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.302786  [   64/60000]
loss: 2.290633  [ 6464/60000]
loss: 2.272354  [12864/60000]
loss: 2.263737  [19264/60000]
loss: 2.250639  [25664/60000]
loss: 2.210253  [32064/60000]
loss: 2.222236  [38464/60000]
loss: 2.190889  [44864/60000]
loss: 2.187336  [51264/60000]
loss: 2.142296  [57664/60000]
Test Error: 
 Accuracy: 48.9%, Avg loss: 2.143976 

Epoch 2
-------------------------------
loss: 2.159092  [   64/60000]
loss: 2.142565  [ 6464/60000]
loss: 2.089532  [12864/60000]
loss: 2.104385  [19264/60000]
loss: 2.050816  [25664/60000]
loss: 1.988085  [32064/60000]
loss: 2.015384  [38464/60000]
loss: 1.941094  [44864/60000]
loss: 1.944069  [51264/60000]
loss: 1.858156  [57664/60000]
Test Error: 
 Accuracy: 60.1%, Avg loss: 1.862945 

Epoch 3
-------------------------------
loss: 1.900129  [   64/60000]
loss: 1.859802  [ 6464/60000]
loss: 1.750313  [12864/60000]
loss: 1.790997  [19264/60000]
loss: 1.674440  [25664/60000]
loss: 1.629329  [32064/600